In [1]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Config, MT5EncoderModel, MT5Tokenizer, Trainer, TrainingArguments
from progeny_tokenizer import TAPETokenizer
import numpy as np
import math
import random
import scipy
import time
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, Dataset, BatchSampler
import typing
from pathlib import Path
import argparse
from collections import OrderedDict
import pickle
import pathlib

import matplotlib.pyplot as plt

In [2]:
noperturb_results_tsv = 'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp01/20iter_temp01-mcmc_seqs.tsv'

In [3]:
perturb_results_tsvs = [
    'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp01/20iter_temp01-mcmc_seqs.tsv',
    'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp001/20iter_temp001-mcmc_seqs.tsv',
    'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_100iter_temp01/100iter_temp01-mcmc_seqs.tsv',
    'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_100iter_temp001/100iter_temp001-mcmc_seqs.tsv',
#     'xxx',
#     'xxx',
#     'xxx',
#     'xxx',
#     'xxx',
#     'xxx',
#     'xxx',
#     'xxx',
#     'xxx',
#     'xxx',
   
]

In [4]:
noperturb_df = pd.read_table(noperturb_results_tsv)

In [5]:
noperturb_df = noperturb_df.sort_values(by='disc_pred', ascending=False)
noperturb_df

,disc_pred,gt_class_pred,MT_edit_dist_vs_WT,generated_seq_ppl,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq,accepted
0,11.440391,3,4,1123.584229,0.727023,1,0.000363,0.727099,The film urines darker moments become smoothed...,0
1,11.423822,2,3,630.396851,0.999945,1,0.000025,0.000030,What you end up getting is the Vertical Bay lo...,1
2,11.420575,3,4,260.160248,0.999677,1,0.000041,0.999689,We gunss and extentkatenția make a convincing ...,0
3,11.415840,3,4,190.767929,0.999758,1,0.000038,0.999771,We gunss and intakekreifen make a convincing c...,0
4,11.407585,2,3,261.530701,0.999917,0,0.000043,0.000039,The movie ultimately relies a bit too heavily ...,0
...,...,...,...,...,...,...,...,...,...,...
25979,-11.070468,1,4,658.821594,0.999882,0,0.999907,0.000026,Can' geometry seem to get anywhere near the me...,1
25980,-11.113492,1,4,730.899170,0.999954,0,0.999961,0.000014,This film was petroleum by and for those folks...,1
25981,-11.158049,1,5,1313.859985,0.999903,0,0.999909,0.000011,This film was petroleum by andtari those folks...,1
25982,-11.263267,1,5,1561.908691,0.999930,0,0.999936,0.000013,This film was petroleum by and Table those fol...,1


In [6]:
print("np.max(noperturb_df['generated_seq_ppl']): ", np.max(noperturb_df['generated_seq_ppl']))
print("np.min(noperturb_df['generated_seq_ppl']): ", np.min(noperturb_df['generated_seq_ppl']))
print("np.mean(noperturb_df['generated_seq_ppl']): ", np.mean(noperturb_df['generated_seq_ppl']))


np.max(noperturb_df['generated_seq_ppl']):  29882510.0
np.min(noperturb_df['generated_seq_ppl']):  5.824474811553955
np.mean(noperturb_df['generated_seq_ppl']):  9645.30751863946


In [7]:
# noperturb_df['sent_delta'] = noperturb_df['gt_class_pred'] - noperturb_df['gen_input_seq_class']
# noperturb_df

# Compute % target class for noperturb

In [8]:
target_classes = [3,4]
topk_list = [10000, 1000, 100, 10]

percent_target_class = []

In [9]:
gt_class_preds = noperturb_df['gt_class_pred']
# gen_input_seq_classes = noperturb_df['gen_input_seq_class']
# sent_deltas = noperturb_df['sent_delta']

In [10]:
df = noperturb_df

# iterate through all perturbed result tsv files

In [11]:
for perturb_results_tsv in perturb_results_tsvs:
    print("*-"*30)
    print("perturb_results_tsv: ", perturb_results_tsv)
    
    perturb_df = pd.read_table(perturb_results_tsv)
    perturb_df = perturb_df.sort_values(by='disc_pred', ascending=False)
#     perturb_df['sent_delta'] = perturb_df['gt_class_pred'] - perturb_df['gen_input_seq_class']

    gt_class_preds = perturb_df['gt_class_pred']
#     gen_input_seq_classes = perturb_df['gen_input_seq_class']
#     sent_deltas = perturb_df['sent_delta']
    generated_seq_ppls = perturb_df['generated_seq_ppl']

    for target_class in target_classes:
        total_num  = len(perturb_df['gt_class_pred'])
        print("target_class: ", target_class)
        num_target_class = np.sum(perturb_df['gt_class_pred'] == target_class)

        percent_target_class = num_target_class / total_num *100
        print("percent_target_class: ", percent_target_class)

        for topk in topk_list:
            topk_gt_class_preds = gt_class_preds[:topk]
#             topk_sent_deltas = sent_deltas[:topk]


            topk_num  = len(topk_gt_class_preds)
            print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
            topk_num_target_class = np.sum(topk_gt_class_preds == target_class)
            topk_percent_target_class = topk_num_target_class / topk_num *100
#             print("topk_num_target_class: ", topk_num_target_class)
#             print("topk_num: ", topk_num)
            print("topk_percent_target_class: ", topk_percent_target_class)

#             topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
#             print("topk_sent_delta_mean: ", topk_sent_delta_mean)

            print("*")
        print("--------------")


    print("-------For all target classes-------")
    print("target_classes: ", target_classes)
    total_num  = len(perturb_df['gt_class_pred'])
    num_target_class = np.sum(perturb_df['gt_class_pred'].isin(target_classes))

    percent_target_class = num_target_class / total_num *100
    print("percent_target_class: ", percent_target_class)

    for topk in topk_list:
        topk_gt_class_preds = gt_class_preds[:topk]
#         topk_sent_deltas = sent_deltas[:topk]
    
        topk_generated_seq_ppls = generated_seq_ppls[:topk]


        topk_num  = len(topk_gt_class_preds)
        print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
        topk_num_target_class = np.sum(topk_gt_class_preds.isin(target_classes))
        topk_percent_target_class = topk_num_target_class / topk_num *100
#         print("topk_num_target_class: ", topk_num_target_class)
#         print("topk_num: ", topk_num)
        print("topk_percent_target_class: ", topk_percent_target_class)

    
        topk_generated_seq_ppl_mean = np.mean(topk_generated_seq_ppls)
        topk_generated_seq_ppl_std = np.std(topk_generated_seq_ppls)
        print("topk_generated_seq_ppl_mean: ", topk_generated_seq_ppl_mean)
        print("topk_generated_seq_ppl_std: ", topk_generated_seq_ppl_std)
        
#         topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
#         print("topk_sent_delta_mean: ", topk_sent_delta_mean)

        print("*")
    
    
    
    
    
        
    # E[% positive, strong-positive] computation

    df = perturb_df

    num_rounds = 100 # N
    round_pool_size = 1000
    topk = 100 # K
    main_pool_size = 25000
    target_classes = [3, 4]

    round_topk = {}
#     cols_to_sort = ['latent_head_pred']
    cols_to_sort = ['disc_pred']
    df_main_pool = df.sample(n=main_pool_size)


    print("--------------")
    print("E[% positive, strong-positive] computation")
    # print("Sorted by ", cols_to_sort)
    for col_to_sort in cols_to_sort:
        print("col_to_sort: ", col_to_sort)
        round_topk[col_to_sort] = {}
        for round_ind in range(num_rounds):
            sampled_rows = df_main_pool.sample(n=round_pool_size)
            sorted_sampled_rows = sampled_rows.sort_values(by=col_to_sort, ascending=False)[:topk]
            topk_rows = sorted_sampled_rows[:topk]
            round_topk[col_to_sort][round_ind] = {}



            for target_class in target_classes:
                total_num  = len(topk_rows['gt_class_pred'])
    #             print("target_class: ", target_class)
                num_target_class = np.sum(topk_rows['gt_class_pred'] == target_class)

                percent_target_class = num_target_class / total_num *100
    #             print("percent_target_class: ", percent_target_class)

                round_topk[col_to_sort][round_ind][target_class] = percent_target_class



    #         print("target_classes: ", target_classes)
            total_num  = len(topk_rows['gt_class_pred'])
            num_target_class = np.sum(topk_rows['gt_class_pred'].isin(target_classes))

            percent_target_class = num_target_class / total_num *100
    #         print("percent_target_class: ", percent_target_class)
            round_topk[col_to_sort][round_ind]['all'] = percent_target_class


        for target_class in target_classes:
            percent_values = []
            for round_ind in range(num_rounds):
                percent_values.append(round_topk[col_to_sort][round_ind][target_class])
            print("target_class: ", target_class)
            mean_percent_values = np.mean(percent_values)
            std_percent_values = np.std(percent_values)
            print("mean_percent_values: ", mean_percent_values)
            print("std_percent_values: ", std_percent_values)



        percent_values = []
        for round_ind in range(num_rounds):
            percent_values.append(round_topk[col_to_sort][round_ind]['all'])
        print("target_classes: ", target_classes)
        mean_percent_values = np.mean(percent_values)
        std_percent_values = np.std(percent_values)
        print("mean_percent_values: ", mean_percent_values)
        print("std_percent_values: ", std_percent_values)

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
perturb_results_tsv:  generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp01/20iter_temp01-mcmc_seqs.tsv
target_class:  3
percent_target_class:  4.375769704433498
topk:  10000
topk_percent_target_class:  8.34
*
topk:  1000
topk_percent_target_class:  16.6
*
topk:  100
topk_percent_target_class:  16.0
*
topk:  10
topk_percent_target_class:  30.0
*
--------------
target_class:  4
percent_target_class:  0.023091133004926108
topk:  10000
topk_percent_target_class:  0.05
*
topk:  1000
topk_percent_target_class:  0.5
*
topk:  100
topk_percent_target_class:  1.0
*
topk:  10
topk_percent_target_class:  0.0
*
--------------
-------For all target classes-------
target_classes:  [3, 4]
percent_target_class:  4.3988608374384235
topk:  10000
topk_percent_target_class:  8.39
topk_generated_seq_ppl_mean:  13918.054308351135
topk_generated_seq_ppl_std:  319639.23372319125
*
topk:  1000
topk_percent_target_class:  17.